In [1]:
# gensim modules
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
import gensim
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from konlpy.utils import pprint
import time
from konlpy.tag import Kkma, Twitter, Komoran
import csv
from scipy import spatial
import pyemd
from datetime import datetime
import pandas as pd
import quandl
import pickle

# numpy
import numpy as np

#정규식
import re

# random
from random import shuffle

C:\Users\rjsfu\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [42]:
#label과 data 읽기
from os import listdir
from os.path import isfile, join

#문서의 data를 quarter별로 저장하기
name = ['현대차', '현대중공업', '한국전력', 'LG디스플레이']
year = ['2008','2009','2010','2011','2012','2013','2014','2015','2016','2017']
quarter = ['1분기', '2분기', '3분기', '4분기']
data = []
for i in name:
    for j in year:
        for k in quarter:
            quar_labels = []
            quar_data = []
            my_path = 'C:/Users/rjsfu/Dropbox/task/텍스트마이닝/data/신문기사/'+i+'/'+j+'/'+k
            for  l in listdir(my_path):
                if l.endswith('.txt'):
                    quar_labels.append(l)
            for doc in quar_labels:
                with open(my_path+ '/' + doc, 'r', encoding = 'utf-8') as f2:
                        document = f2.read()
                        document = re.sub('[a-zA-Z]', '', document)
                        document = re.sub('[【】◆•◇ⓒ○→*■ㆍ\t\n\{\}\[\]\/?,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                                          '', document)
                        quar_data.append(document)
                        f2.close()
            data.append(quar_data)

In [38]:
#분리된 token pickle 파일로 저장하기
out_path = "C:/Users/rjsfu/Dropbox/task/텍스트마이닝/pickle data"

out_file = out_path + "/four_corp_data.pkl"
with open(out_file, 'wb') as f:
    pickle.dump(data, f)

In [2]:
with open('C:/Users/rjsfu/Dropbox/task/텍스트마이닝/pickle data/four_corp_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [3]:
#문서를 형태소 분리
tagger = Twitter()
tokens = []
tag = ['Noun', 'Advjective']
process_time= time.time()
for quar_data in data:
    tokens_quar = []
    for text in quar_data:
        article = []
        tagged = tagger.pos(text)
        for word, pos in tagged:
            if pos in tag:
                article.append(word)
        tokens_quar.append(article)
    tokens.append(tokens_quar)
process_time = time.time() - process_time
print('%.3f secs' % (process_time))

#한글자 지우기
remove_one = []
for i in tokens:
    quar_remove_one = []
    for j in i:
        article_remove_one = []
        for k in j:
            if len(k) == 1:
                pass
            else:
                article_remove_one.append(k)
        quar_remove_one.append(article_remove_one)
    remove_one.append(quar_remove_one)

C:\Users\rjsfu\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


469.694 secs


In [4]:
#분리된 remove_one pickle 파일로 저장하기
out_path = "C:/Users/rjsfu/Dropbox/task/텍스트마이닝/pickle data"

out_file = out_path + "/four_corp_remoce_one.pkl"
with open(out_file, 'wb') as f:
    pickle.dump(data, f)

In [7]:
#감성사전 불러오기
with open('C:/Users/rjsfu/Dropbox/task/텍스트마이닝/pickle data/sentiment_dict_article.pkl', 'rb') as f:
    sentiment_dict = pickle.load(f)

In [25]:
len(sentiment_dict['neutral'])

4158

In [8]:
#기업별 분기별 데이터 나누기
data_현대차 = remove_one[:40]
data_현대중공업 = remove_one[40:80]
data_한국전력 = remove_one[80:120]
data_LG디스플레이 = remove_one[120:160]

In [9]:
#현대차 분기의 마지막 10기사의 점수
point = []

for i in data_현대차: #한개의 분기
    quar_point = []
    reverse = i[::-1][:9]
    for j in reverse:
        pos = 0
        neg = 0
        neu = 0
        for k in j:
            if k in sentiment_dict['positive']:
                pos += 1
            elif k in sentiment_dict['negative']:
                neg += 1
            elif k in sentiment_dict['neutral']:
                neu += 1
            else:
                 pass
        if pos == 0 and neg == 0:
            quar_point.append(0)
        else:
            art_point = (pos-neg)/(pos+neg)
            quar_point.append(art_point)
    point.append(quar_point)
    
mean_point_현대차 = []

for i in point:
    if len(i) == 0:
        mean_point_현대차.append(0)
    else:
        mean_point_현대차.append(sum(i, 0.0)/len(i))

In [11]:
#현대중공업 분기의 마지막 10기사의 점수
point = []

for i in data_현대중공업: #한개의 분기
    quar_point = []
    reverse = i[::-1][:9]
    for j in reverse:
        pos = 0
        neg = 0
        neu = 0
        for k in j:
            if k in sentiment_dict['positive']:
                pos += 1
            elif k in sentiment_dict['negative']:
                neg += 1
            elif k in sentiment_dict['neutral']:
                neu += 1
            else:
                 pass
        if pos == 0 and neg == 0:
            quar_point.append(0)
        else:
            art_point = (pos-neg)/(pos+neg)
            quar_point.append(art_point)
    point.append(quar_point)
    
mean_point_현대중공업 = []

for i in point:
    if len(i) == 0:
        mean_point_현대중공업.append(0)
    else:
        mean_point_현대중공업.append(sum(i, 0.0)/len(i))

In [12]:
#한국전력 분기의 마지막 10기사의 점수
point = []

for i in data_한국전력: #한개의 분기
    quar_point = []
    reverse = i[::-1][:9]
    for j in reverse:
        pos = 0
        neg = 0
        neu = 0
        for k in j:
            if k in sentiment_dict['positive']:
                pos += 1
            elif k in sentiment_dict['negative']:
                neg += 1
            elif k in sentiment_dict['neutral']:
                neu += 1
            else:
                 pass
        if pos == 0 and neg == 0:
            quar_point.append(0)
        else:
            art_point = (pos-neg)/(pos+neg)
            quar_point.append(art_point)
    point.append(quar_point)
    
mean_point_한국전력 = []

for i in point:
    if len(i) == 0:
        mean_point_한국전력.append(0)
    else:
        mean_point_한국전력.append(sum(i, 0.0)/len(i))

In [13]:
#LG디스플레이 분기의 마지막 10기사의 점수
point = []

for i in data_LG디스플레이: #한개의 분기
    quar_point = []
    reverse = i[::-1][:9]
    for j in reverse:
        pos = 0
        neg = 0
        neu = 0
        for k in j:
            if k in sentiment_dict['positive']:
                pos += 1
            elif k in sentiment_dict['negative']:
                neg += 1
            elif k in sentiment_dict['neutral']:
                neu += 1
            else:
                 pass
        if pos == 0 and neg == 0:
            quar_point.append(0)
        else:
            art_point = (pos-neg)/(pos+neg)
            quar_point.append(art_point)
    point.append(quar_point)
    
mean_point_LG디스플레이 = []

for i in point:
    if len(i) == 0:
        mean_point_LG디스플레이.append(0)
    else:
        mean_point_LG디스플레이.append(sum(i, 0.0)/len(i))

In [14]:
#사업보고서 감성점수 불러오기
my_path = 'C:/Users/rjsfu/Dropbox/task/텍스트마이닝/관련 자료'

with open(my_path + '/' + 'report_sentiscore' +'.csv', 'r', encoding = 'utf-8') as f:
    doc = f.read()
    report_sent = doc.split('\n')

In [15]:
report_sent_LG디스플레이 = report_sent[1:41]
report_sent_현대차 = report_sent[41:81]
report_sent_현대중공업 = report_sent[81:121]
report_sent_한국전력 = report_sent[121:]

report_score_현대차 = []
for i in report_sent_현대차:
    temp = i.split(',')
    report_score_현대차.append(temp[5])
    
report_score_현대중공업 = []
for i in report_sent_현대중공업:
    temp = i.split(',')
    report_score_현대중공업.append(temp[5])
    
report_score_LG디스플레이 = []
for i in report_sent_LG디스플레이:
    temp = i.split(',')
    report_score_LG디스플레이.append(temp[5])


report_score_한국전력 = []
for i in report_sent_한국전력:
    temp = i.split(',')
    report_score_한국전력.append(temp[5])

IndexError: list index out of range

In [17]:
#주가의 up& down 불러오기
my_path = 'C:/Users/rjsfu/Dropbox/task/텍스트마이닝/관련 자료'

#######한국전력###########
with open(my_path + '/' + '한국전력 up&down' +'.csv', 'r', encoding = 'utf-8') as f:
    doc = f.read()
    한국전력_up_down = doc.split('\n')

한국전력_ud = []
for i in 한국전력_up_down:
    temp = i.split(',')
    한국전력_ud.append(temp[1])
    
for i in range(len(한국전력_ud)):
    if 한국전력_ud[i] == '2':
        한국전력_ud[i] = 0
    elif 한국전력_ud[i] == '1':
        한국전력_ud[i] = 1
    else:
        한국전력_ud[i] = 0 
    
    
    
    
######현대중공업##########    
with open(my_path + '/' + '현대중공업 up&down' +'.csv', 'r', encoding = 'utf-8') as f:
    doc = f.read()
    현대중공업_up_down = doc.split('\n')
    
현대중공업_ud = []
for i in 현대중공업_up_down:
    temp = i.split(',')
    현대중공업_ud.append(temp[1])
    
for i in range(len(현대중공업_ud)):
    if 현대중공업_ud[i] == '2':
        현대중공업_ud[i] = 0
    elif 현대중공업_ud[i] == '1':
        현대중공업_ud[i] = 1
    else:
        현대중공업_ud[i] = 0 
    
    
#######현대차##########
with open(my_path + '/' + '현대차 up&down' +'.csv', 'r', encoding = 'utf-8') as f:
    doc = f.read()
    현대차_up_down = doc.split('\n')
    
현대차_ud = []
for i in 현대차_up_down:
    temp = i.split(',')
    현대차_ud.append(temp[1])

for i in range(len(현대차_ud)):
    if 현대차_ud[i] == '2':
        현대차_ud[i] = 0
    elif 현대차_ud[i] == '1':
        현대차_ud[i] = 1
    else:
        현대차_ud[i] = 0

        
        
        
########LG디스플레이########
with open(my_path + '/' + 'LG디스플레이 up&down' +'.csv', 'r', encoding = 'utf-8') as f:
    doc = f.read()
    LG디스플레이_up_down = doc.split('\n')
    
LG디스플레이_ud = []
for i in LG디스플레이_up_down:
    temp = i.split(',')
    LG디스플레이_ud.append(temp[1])
    
for i in range(len(LG디스플레이_ud)):
    if LG디스플레이_ud[i] == '2':
        LG디스플레이_ud[i] = 0
    elif LG디스플레이_ud[i] == '1':
        LG디스플레이_ud[i] = 1
    else:
        LG디스플레이_ud[i] = 0

In [20]:
#현대차 정확도
final_score_현대차 = []
for i in range(len(report_score_현대차)):
    score = (0.7*mean_point_현대차[i]) + (0.3*float(report_score_현대차[i]))
    final_score_현대차.append(score)
    
score_ud_현대차 = []
for i in final_score_현대차:
    if i > 0.25:
        score_ud_현대차.append(1)
    else:
        score_ud_현대차.append(0)
        
acc_현대차 = np.mean(np.equal(np.array(score_ud_현대차), np.array(현대차_ud)))
acc_현대차

0.55

In [14]:
#LG디스플레이 정확도
final_score_LG디스플레이 = []
for i in range(len(report_score_LG디스플레이)):
    score = (0.7*mean_point_LG디스플레이[i]) + (0.3*float(report_score_LG디스플레이[i]))
    final_score_LG디스플레이.append(score)
    
score_ud_LG디스플레이 = []
for i in final_score_LG디스플레이:
    if i > 0.45:
        score_ud_LG디스플레이.append(1)
    else:
        score_ud_LG디스플레이.append(0)
        
acc_LG디스플레이 = np.mean(np.equal(np.array(score_ud_LG디스플레이), np.array(LG디스플레이_ud)))
acc_LG디스플레이

0.65

In [35]:
#현대중공업 정확도
final_score_현대중공업 = []
for i in range(len(report_score_현대중공업)):
    score = (0.7*mean_point_현대중공업[i]) + (0.3*float(report_score_현대중공업[i]))
    final_score_현대중공업.append(score)
    
score_ud_현대중공업 = []
for i in final_score_현대중공업:
    if i > 0.22:
        score_ud_현대중공업.append(1)
    else:
        score_ud_현대중공업.append(0)
        
acc_현대중공업 = np.mean(np.equal(np.array(score_ud_현대중공업), np.array(현대중공업_ud)))
acc_현대중공업

0.55

In [32]:
#한국전력 정확도
final_score_한국전력 = []
for i in range(len(report_score_한국전력)):
    score = (0.7*mean_point_한국전력[i]) + (0.3*float(report_score_한국전력[i]))
    final_score_한국전력.append(score)
    
score_ud_한국전력 = []
for i in final_score_한국전력:
    if i > 0.45:
        score_ud_한국전력.append(1)
    else:
        score_ud_한국전력.append(0)
        
acc_한국전력 = np.mean(np.equal(np.array(score_ud_한국전력), np.array(한국전력_ud)))
acc_한국전력

0.675